#Loading dataset & installation

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.3 MB/s 
     |████████████████████████████████| 182 kB 61.3 MB/s 
     |████████████████████████████████| 7.6 MB 65.8 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# importing required libraries
import pandas as pd
import numpy as np
import torch

In [ ]:
# loading the dataset
df_eval = pd.read_csv('Testset20.csv')
df_eval.head()

,Topic,Text,Question,Answer
0,The Universe,"The Universe is everything we can touch, feel,...",Where is the edge of the Universe?,we cannot see the edge
1,The Universe,"The Universe is everything we can touch, feel,...",What materials are filled in the Space?,Space is also filled with radiation (e.g. ligh...
2,The Universe,"The Universe is everything we can touch, feel,...",What is the speed of light?,"300,000 km per second"
3,Our nearest star,The Sun is our nearest star. The Sun provides ...,Which direction is the Earth spinning?,Earth is spinning towards the east
4,Our nearest star,The Sun is our nearest star. The Sun provides ...,how delayed the sun light to reach earth?,it takes about 8 minutes for sunlight to reach us


In [ ]:
import string
import re

def normalize_text(text):
    text = text.lower()
    text = "".join(ch for ch in text if ch not in set(string.punctuation))
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    text = re.sub(regex, " ", text)
    text = " ".join(text.split())
    return text
'''
test = "Wipe ##s out life over much of the planet"
print(normalize_text(test))
'''

'\ntest = "Wipe ##s out life over much of the planet"\nprint(normalize_text(test))\n'

In [ ]:
#Testinng
'''
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
import torch

from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

name = 'distilbert-base-uncased-distilled-squad'

tokenizer = AutoTokenizer.from_pretrained(name)
model = AutoModelForQuestionAnswering.from_pretrained(name)
question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"

inputs = tokenizer(question, text, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

print('answer_start_index', answer_start_index)
print('answer_end_index', answer_end_index)

predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
result = tokenizer.decode(predict_answer_tokens)
print(result)
'''

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/451 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/265M [00:00<?, ?B/s]

answer_start_index tensor(10)
answer_end_index tensor(12)
a nice puppet


In [ ]:
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
import torch
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased-distilled-squad')
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased-distilled-squad')

#testinng
question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"

inputs = tokenizer(question, text, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

answer_start_index = torch.argmax(outputs.start_logits)
answer_end_index = torch.argmax(outputs.end_logits)

predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)


'a nice puppet'

In [ ]:
saved_model = torch.load('/content/drive/MyDrive/BERT-SQuAD/auto_distilbert.pth',map_location='cpu')
saved_model.eval()

In [ ]:
#Testing saved model
question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"

saved_model.eval()
inputs = tokenizer(question, text, return_tensors="pt")
with torch.no_grad():
    outputs = saved_model(**inputs)

answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

print('answer_start_index', answer_start_index)
print('answer_end_index', answer_end_index)

predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
result = tokenizer.decode(predict_answer_tokens)
print(result)

answer_start_index tensor(10)
answer_end_index tensor(12)
a nice puppet


In [ ]:
after_predicted_answers = []
test_df = df_eval

for i, row in test_df.iterrows():
  question = row['Question']
  text = row['Text']
  input_ids = tokenizer.encode(question, text)
  tokens = tokenizer.convert_ids_to_tokens(input_ids)
  output = saved_model(torch.tensor([input_ids]))
  
  answer_start = torch.argmax(output.start_logits)
  answer_end = torch.argmax(output.end_logits)
  if answer_end >= answer_start:
    answer = " ".join(tokens[answer_start:answer_end+1])
  else:
    print("I am unable to find the answer to this question.")
    answer = " "

  #print("Text:\n{}".format(row["Text"]))
  print("\n *** Question:\n{}".format(question.capitalize()))
  print("\nPredicted Answer:\n{}.".format(answer.capitalize()))
  normal_answer = normalize_text(answer)
  # print("\nNormalized Predicted Answer:\n{}.".format(normal_answer.capitalize()))
  print('\nTruth Answer:\n', row['Answer'])

  after_predicted_answers.append(normal_answer.capitalize())

dict = {'after_predicted_answers': after_predicted_answers}

# Creating Dataframe
df_result = pd.DataFrame(dict)

# Export DataFrame to CSV
df_result.to_csv('df_result_after_predicted_answers.csv')


I am unable to find the answer to this question.

 *** Question:
Where is the edge of the universe?

Predicted Answer:
 .

Truth Answer:
 we cannot see the edge

 *** Question:
What materials are filled in the space?

Predicted Answer:
Living things , planets , stars , galaxies , dust clouds , light , and even time . before the birth of the universe , time , space and matter did not exist . the universe contains billions of galaxies , each containing millions or billions of stars . the space between the stars and galaxies is largely empty . however , even places far from stars and planets contain scattered particles of dust or a few hydrogen atoms per cubic cent ##imeter . space is also filled with radiation ( e . g . light and heat ) , magnetic fields and high energy particles ( e . g . cosmic rays ).

Truth Answer:
 Space is also filled with radiation (e.g. light and heat), magnetic fields and high energy particles (e.g. cosmic rays)

 *** Question:
What is the speed of light?

Predi

# Comparison before/after finetuninng from saved model

Before

In [ ]:
ori_model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased-distilled-squad')

In [ ]:
before_predicted_answers = []

for i, row in test_df.iterrows():
  question = row['Question']
  text = row['Text']
  input_ids = tokenizer.encode(question, text)
  tokens = tokenizer.convert_ids_to_tokens(input_ids)
  
  output = ori_model(torch.tensor([input_ids]))
  answer_start = torch.argmax(output.start_logits)
  answer_end = torch.argmax(output.end_logits)
  if answer_end >= answer_start:
    answer = " ".join(tokens[answer_start:answer_end+1])
  else:
    answer = " "
    print("----- I am unable to find the answer to this question.")

  #print("Text:\n{}".format(row["Text"]))
  print("\n###Question:\n{}".format(question.capitalize()))
  print("\nPredicted Answer:\n{}.".format(answer.capitalize()))
  normal_answer = normalize_text(answer)
  print("\nNormalized Predicted Answer:\n{}.".format(normal_answer.capitalize()))
  print('\nTruth Answer:\n', row['Answer'])

  before_predicted_answers.append(normal_answer.capitalize())

dict = {'before_predicted_answers': before_predicted_answers}

# Creating Dataframe
df_result = pd.DataFrame(dict)

# Export DataFrame to CSV
df_result.to_csv('df_result_before_predicted_answers.csv')


----- I am unable to find the answer to this question.

###Question:
Where is the edge of the universe?

Predicted Answer:
 .

Normalized Predicted Answer:
.

Truth Answer:
 we cannot see the edge

###Question:
What materials are filled in the space?

Predicted Answer:
Radiation ( e . g . light and heat ) , magnetic fields and high energy particles.

Normalized Predicted Answer:
Radiation e g light and heat magnetic fields and high energy particles.

Truth Answer:
 Space is also filled with radiation (e.g. light and heat), magnetic fields and high energy particles (e.g. cosmic rays)

###Question:
What is the speed of light?

Predicted Answer:
300 , 000 km per second.

Normalized Predicted Answer:
300 000 km per second.

Truth Answer:
 300,000 km per second

###Question:
Which direction is the earth spinning?

Predicted Answer:
East.

Normalized Predicted Answer:
East.

Truth Answer:
 Earth is spinning towards the east

###Question:
How delayed the sun light to reach earth?

Predicted A

In [ ]:
import pandas as pd

report_before = pd.read_csv('df_result_before_predicted_answers.csv')
report_after = pd.read_csv('df_result_after_predicted_answers.csv')

df_combine = pd.concat([report_before,report_after, df_eval['Answer']], axis=1)
df_combine.drop(columns='Unnamed: 0')

df_combine.head()

,Unnamed: 0,before_predicted_answers,Unnamed: 0,after_predicted_answers,Answer
0,0,NaN,0,NaN,we cannot see the edge
1,1,Radiation e g light and heat magnetic fields a...,1,Living things planets stars galaxies dust clou...,Space is also filled with radiation (e.g. ligh...
2,2,300 000 km per second,2,300 000 km per second,"300,000 km per second"
3,3,East,3,Towards east,Earth is spinning towards the east
4,4,8 minutes,4,About 8 minutes,it takes about 8 minutes for sunlight to reach us


In [ ]:
df_combine = df_combine.drop(columns='Unnamed: 0')
df_combine['before_predicted_answers'].fillna('', inplace=True)
df_combine['after_predicted_answers'].fillna('', inplace=True)

df_combine.to_csv('df_combine.csv')
df_combine.head()

In [ ]:
df_combine.head(20)

,before_predicted_answers,after_predicted_answers,Answer
0,,,we cannot see the edge
1,Radiation e g light and heat magnetic fields a...,Living things planets stars galaxies dust clou...,Space is also filled with radiation (e.g. ligh...
2,300 000 km per second,300 000 km per second,"300,000 km per second"
3,East,Towards east,Earth is spinning towards the east
4,8 minutes,About 8 minutes,it takes about 8 minutes for sunlight to reach us
5,Spec tro graphs,Spec tro graphs,Special instruments known as spectrographs tel...
6,X rays,,"This finds its way to the surface as visible, ..."
7,Some stars are much bigger and hotter,Some stars are much bigger and hotter than sun,Some stars are much bigger and hotter than the...
8,Ant ares,Red super gia nt ant ares,The red supergiant Antares is 800 times wider ...
9,Blue white,Blue white,Blue-white stars


#EVAL

## BERTScore

In [3]:
!pip install evaluate
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 1.2 MB/s 
     |████████████████████████████████| 182 kB 11.0 MB/s 
     |████████████████████████████████| 132 kB 68.3 MB/s 
     |████████████████████████████████| 451 kB 92.5 MB/s 
     |████████████████████████████████| 212 kB 90.2 MB/s 
     |████████████████████████████████| 127 kB 70.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60 kB 3.3 MB/s 
     |████████████████████████████████| 5.5 MB 10.7 MB/s 
     |████████████████████████████████| 7.6 MB 84.0 MB/s 


In [ ]:
import pandas as pd

df_result = pd.read_csv('/content/yoonjung_distillbert_prediction.csv')
df_result.head()
df_result['after_predicted_answers'] = df_result['after_predicted_answers'].fillna("", inplace=True)

In [4]:

from evaluate import load
bertscore = load("bertscore")
predictions = df_result['after_predicted_answers'].values.tolist()
references = df_result['Answer'].values.tolist()
results = bertscore.compute(predictions=predictions, references=references, lang="en")
print(results)
print(sum(results['f1']) / len(results['f1']))

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

{'precision': [0.0, 0.8217209577560425, 0.9804799556732178, 0.8637048602104187, 0.9095813035964966, 0.8127009272575378, 0.0, 0.9846529364585876, 0.8111889362335205, 0.9465489983558655, 0.8421709537506104, 0.7884202003479004, 0.9814007878303528, 0.9655818939208984, 0.9332159757614136, 0.9110208749771118, 0.7758268713951111, 0.8995575904846191, 0.8378133177757263, 0.9475514888763428], 'recall': [0.0, 0.9002051949501038, 0.9638479948043823, 0.8969683647155762, 0.8645907044410706, 0.8070739507675171, 0.0, 0.9823421835899353, 0.8299635052680969, 0.9237886071205139, 0.8457452058792114, 0.9123310446739197, 0.9815690517425537, 0.9644895195960999, 0.9001163244247437, 0.8587921261787415, 0.8285014033317566, 0.8278716206550598, 0.8655683994293213, 0.8873205184936523], 'f1': [0.0, 0.8591744303703308, 0.9720928072929382, 0.880022406578064, 0.8865154981613159, 0.8098777532577515, 0.0, 0.9834962487220764, 0.8204689025878906, 0.9350302815437317, 0.8439542651176453, 0.8458617329597473, 0.98148494958877

## BLEU

In [ ]:
gt_answer = df_combine['Answer'].values.tolist()
gt_answer
gt_answer = list(map(lambda x: x.lower(), gt_answer))
print(gt_answer)

after_answer = df_combine['after_predicted_answers'].values.tolist()
after_answer = list(map(lambda x: x.lower(), after_answer))
print(after_answer)

['we cannot see the edge', 'space is also filled with radiation (e.g. light and heat), magnetic fields and high energy particles (e.g. cosmic rays)', '300,000 km per second', 'earth is spinning towards the east', 'it takes about 8 minutes for sunlight to reach us', 'special instruments known as spectrographs tell us that the sun', 'this finds its way to the surface as visible, infrared and ultraviolet light, and x-rays', 'some stars are much bigger and hotter than the sun', 'the red supergiant antares is 800 times wider than the sun.', 'blue-white stars', 'the largest galaxies look like squashed balls', 'they contain up to 10 million million stars', 'galaxies were born only a few hundred million years after the universe was created', 'every object in the universe attracts every other object in the universe', 'the strength of gravity depends on the size and density of an object', "hanks to visiting spacecraft, we know that the planet's appearance is due to rust in the martian rocks.", '

In [ ]:
!pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 118 kB 5.0 MB/s 


In [ ]:
with open("file_gt", 'w') as output:
    for row in gt_answer:
        output.write(str(row) + '\n')

In [ ]:
with open("file_pred", 'w') as output:
    for row in after_answer:
        output.write(str(row) + '\n')

In [ ]:
!sacrebleu file_gt < file_pred

{
 "name": "BLEU",
 "score": 21.0,
 "signature": "nrefs:1|case:mixed|eff:no|tok:13a|smooth:exp|version:2.3.1",
 "verbose_score": "34.5/24.5/18.7/12.3 (BP = 1.000 ratio = 1.041 hyp_len = 255 ref_len = 245)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "13a",
 "smooth": "exp",
 "version": "2.3.1"
}


## F1 tokens

In [5]:
def compute_f1(prediction, truth):
    pred_tokens = prediction.split()
    truth_tokens = truth.split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

In [6]:
df_result.head()

,Unnamed: 0,before_predicted_answers,after_predicted_answers,Answer
0,0,NaN,,we cannot see the edge
1,1,Radiation e g light and heat magnetic fields a...,Living things planets stars galaxies dust clou...,Space is also filled with radiation (e.g. ligh...
2,2,300 000 km per second,300 000 km per second,"300,000 km per second"
3,3,East,Towards east,Earth is spinning towards the east
4,4,8 minutes,About 8 minutes,it takes about 8 minutes for sunlight to reach us


In [7]:
f1_list = []

for i, row in df_result.iterrows() :
  pred = row['after_predicted_answers']
  gt = row['Answer']
  f1_list.append(compute_f1(pred, gt))

f1_list
print(sum(f1_list) / len(f1_list))

0.2664903808318258
